In [ ]:
import os
import sys
import pandas as pd
import numpy as np
import torch

In [ ]:
from torch.utils.data import Dataset, DataLoader

# Pytorch DataLoader
Pytorch는 데이터를 전처리하고 배치화할 수 있는 클래스를 제공한다.    
`Dataset` 클래스는 데이터를 **전처리**하고 dictionary 또는 list 타입으로 변경할 수 있다.   
`DataLoader` 클래스는 데이터 **1. 셔플 2. 배치화 3. 멀티 프로세스** 기능을 제공한다. 

[OFFICAL DOCUMENT](https://pytorch.org/docs/stable/data.html#torch.utils.data.DataLoader)

# Table of Contents
- [Dataset](#Dataset)
- [Dataloader](#DataLoader)


## Dataset
- 모든 custom dataset 클래스는 `Dataset()` 클래스를 상속받아야 함.
- `__getitem__()`와 `__len__()` 메소드를 반드시 오버라이딩해야 함. 
- `DataLoader` 클래스가 배치를 만들 때 `Dataset` 인스턴스의 `__getitem__()` 메소드를 사용해 데이터에 접근함
- 해당 Dataset 클래스는 string sequence 데이터를 **tokenize** & **tensorize**한다. 

In [ ]:
# !pip install transformers
from torchtext.datasets import AG_NEWS
from transformers import BertTokenizer, BertModel
from typing import Iterator

In [ ]:
trainer_iter = AG_NEWS(split = 'train')

In [ ]:
try:
    trainer_iter[0]
except NotImplementedError:
    print(f"__getitem__() function not implemented.")

In [ ]:
next(trainer_iter)

In [ ]:
class Custom_Dataset(Dataset):

    def __init__(self, data: Iterator):
        self.tokenizer = BertTokenizer.from_pretrained("bert-base-cased")
        self.target = []
        self.text = []
        for target, text in data:
            self.target.append(target)
            self.text.append(text)
  
    def __len__(self):
        return len(self.target)

    def __getitem__(self, index):
        # encode
        token_ids = self.tokenizer.encode(
        text = self.text[index],
        truncation = True,
        )
        
        # tensorize
        return torch.tensor(token_ids), torch.tensor([self.target[index]])




In [ ]:
train_dataset = Custom_Dataset(trainer_iter)

In [ ]:
train_dataset[0]

In [ ]:
len(train_dataset)

In [ ]:
# decode to see original text
train_dataset.tokenizer.decode(train_dataset[0][0])

## Dataloader
- `dataset`
    - **map-style** dataset
    (`Dataset`)
    - iterable style dataset
      - `__iter__()`
- `batch_size` 
  - int
- `shuffle`
  - bool
- `sampler`
  - data index 이터레이터
- `collate_fn`